In [8]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

%cd Neural-Cellular-Automata

Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 74 (delta 43), reused 34 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 146.66 KiB | 5.43 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/Neural-Cellular-Automata/Neural-Cellular-Automata


In [ ]:
!pip install -U gdown

file_ids = ['1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj',
            '1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG',
            '1PqrOpn4NJF3xp4Jw2KtRRVD_20Feucsc',
            '1DtZJ5eD2PV4sO51MMQy9CVTZxgBKj_Gx']

file_names = ['pathmnist.npz', 'pathmnist_64.npz', 'pathmnist_128.npz', 'pathmnist_224.npz']

for file_id, file_name in zip(file_ids, file_names):
    !gdown --id {file_id} --output {file_name}

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 4, in <module>
    from gdown.__main__ import main
  File "/usr/local/lib/python3.11/dist-packages/gdown/__init__.py", line 6, in <module>
    from .cached_download import cached_download
  File "/usr/local/lib/python3.11/dist-packages/gdown/cached_download.py", line 12, in <module>
    from .download import download
  File "/usr/local/lib/python3.11/dist-packages/gdown/download.py", line 13, in <module>
    import bs4
  File "/usr/local/lib/python3.11/dist-packages/bs4/__init__.py", line 64, in <module>
    from .builder import (
  File "/usr/local/lib/python3.11/dist-packages/bs4/builder/__init__.py", line 843, in <module>
    from . import _lxml
  File "/usr/local/lib/python3.11/dist-packages/bs4/builder/_lxml.py", line 29, in <module>
    from lxml import etree
  File "<frozen importlib._bootstrap>", line 405, in parent
KeyboardInterrupt
^C
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWar

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!pip install -q medmnist scikit-learn

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from medmnist import PathMNIST
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, mean_absolute_error
import seaborn as sns
from models import NCA, CNNBaseline


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.6 MB/s eta 0:00:00
Using device: cuda


In [19]:
from google.colab import drive
import os
import shutil

drive_folder = "/content/Neural-Cellular-Automata/"
cache_dir = os.path.expanduser("~/.medmnist")
os.makedirs(cache_dir, exist_ok=True)

resolutions = ["", "_64", "_128", "_224"]
for res in resolutions:
    filename = f"pathmnist{res}.npz"
    src = os.path.join(drive_folder, filename)
    dst = os.path.join(cache_dir, filename)

    if os.path.exists(src):
        shutil.copyfile(src, dst)
        print(f"Copied {filename} to cache.")
    else:
        print(f"File not found in Drive: {filename}")

KeyboardInterrupt: 

In [13]:
nca = NCA().to(device)
nca.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_nca_pathmnist.pth"))
nca.eval()

cnn = CNNBaseline().to(device)
cnn.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_cnn_pathmnist.pth"))
cnn.eval()

CNNBaseline(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
  )
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=128, out_features=9, bias=True)
)

In [14]:
def get_loader(size, batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = PathMNIST(split="test", size=size, download=False, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [15]:
@torch.no_grad()
def evaluate(model, loader, name="Model", size=28, save_dir="/content/drive/MyDrive/NCA/results", is_NCA=False):
    all_preds, all_labels = [], []

    for x, y in loader:
        x, y = x.to(device), y.squeeze()
        if is_NCA:
            out, rgb_steps = model(x, True)
        else:
            out = model(x)
        pred = out.argmax(dim=1).cpu().numpy()
        label = y.numpy()
        all_preds.extend(pred)
        all_labels.extend(label)

    # Metrics
    cm = confusion_matrix(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)

    # Logging
    print(f"\n{name} @ {size}x{size}")
    print("Balanced Accuracy:", f"{bal_acc:.4f}")
    print("Mean Absolute Error (MAE):", f"{mae:.4f}")
    print(report)

    # Save confusion matrix
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix: {name} @ {size}x{size}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    fname = f"{save_dir}/cm_{name.replace(' ', '_')}_{size}x{size}.png"
    plt.savefig(fname)
    plt.close()
    print(f"Confusion matrix saved to: {fname}")

In [16]:
for size in [28, 64, 128, 224]:
    print(f"\n==============================")
    print(f"Resolution: {size}x{size}")
    loader = get_loader(size)

    print("CNN:")
    evaluate(cnn, loader, name="CNN", size=size)

    print("NCA:")
    evaluate(nca, loader, name="NCA", size=size, is_NCA=True)


Resolution: 28x28
CNN:

CNN @ 28x28
Balanced Accuracy: 0.4015
Mean Absolute Error (MAE): 1.9591
              precision    recall  f1-score   support

           0     1.0000    0.0919    0.1684      1338
           1     0.8777    1.0000    0.9349       847
           2     0.3106    0.7375    0.4371       339
           3     0.0000    0.0000    0.0000       634
           4     0.3677    0.9894    0.5361      1035
           5     0.6000    0.0963    0.1659       592
           6     0.3621    0.1134    0.1727       741
           7     0.0851    0.0095    0.0171       421
           8     0.3347    0.5758    0.4234      1233

    accuracy                         0.4316      7180
   macro avg     0.4375    0.4015    0.3173      7180
weighted avg     0.5069    0.4316    0.3448      7180

Confusion matrix saved to: /content/drive/MyDrive/NCA/results/cm_CNN_28x28.png
NCA:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



NCA @ 28x28
Balanced Accuracy: 0.2798
Mean Absolute Error (MAE): 1.8765
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1338
           1     0.3238    0.9988    0.4890       847
           2     0.8778    0.4661    0.6089       339
           3     0.0000    0.0000    0.0000       634
           4     0.5315    0.1710    0.2588      1035
           5     0.2188    0.0236    0.0427       592
           6     0.0016    0.0013    0.0015       741
           7     0.1682    0.6105    0.2637       421
           8     0.1655    0.2466    0.1980      1233

    accuracy                         0.2447      7180
   macro avg     0.2541    0.2798    0.2070      7180
weighted avg     0.2127    0.2447    0.1769      7180

Confusion matrix saved to: /content/drive/MyDrive/NCA/results/cm_NCA_28x28.png

Resolution: 64x64


RuntimeError: Dataset not found.  You can set `download=True` to download it